# **Indexación y selección** 

La indexación y selección son herramientas esenciales para acceder, filtrar y manipular datos de manera eficiente. Permiten optimizar el rendimiento al trabajar con grandes volúmenes de información, aplicar transformaciones de forma precisa y mejorar la velocidad en análisis de datos, incluyendo datos geoespaciales. Estas técnicas facilita la exploración y el procesamiento de datos, haciendo que nuestras tareas sean más rápidas y estructuradas.

## **1. Operadores de indexación**

### **1.1. Selección de columnas**


```{admonition} Seleccionar una única columna
:class: tip

#### **`df[col1]`**
```

```{admonition} Seleccionar múltiples columnas
:class: tip

#### **`df[[col1,col2,col3]]`**
```

### **1.2. Selección por máscara**

```{admonition} Seleccionar por máscara
:class: tip

#### **`df[<expression>]`**

Donde <expression> es una expresión boleana que retorna `True` o `False`
```

## **2. Selección por Etiqueta**

### **2.1. Selección de filas**

```{admonition} Seleccionar una única fila
:class: tip

#### **`df.loc[row1]`**
```

```{admonition} Seleccionar múltiples filas
:class: tip

#### **`df.loc[[row1, row2, row3]]`**
```

```{admonition} Seleccionar con slicing
:class: tip

#### **`df.loc[rowA:rowN]`**
```

### **2.2. Selección de columnas**

```{admonition} Seleccionar una única columna
:class: tip

#### **`df.loc[:, col1]`**
```

```{admonition} Seleccionar múltiples columnas
:class: tip

#### **`df.loc[:, [col1, col2, col3]]`**
```

```{admonition} Seleccionar con slicing
:class: tip

#### **`df.loc[:, colA:colN]`**
```

### **2.3. Selección de filas y columnas**

```{admonition} Seleccionar una fila y una columna
:class: tip

#### **`df.loc[row1, col1]`**
```

```{admonition} Seleccionar múltiples filas y columnas
:class: tip

#### **`df.loc[[row1, row2, row3], [col1, col2, col3]]`**
```

```{admonition} Seleccionar con Slicing
:class: tip

#### **`df.loc[rowA:rowN, colA:colN]`**
```

### **2.4. Selección por máscara**

```{admonition} Seleccionar por máscara
:class: tip

#### **`df.loc[<expression>]`**

Donde <expression> es una condición booleana aplicada sobre el DataFrame, evaluando cada fila y devolviendo True para las filas que se seleccionarán.
```

### **2.5. Actualización de valores**

```{admonition} Actualización de un valor
:class: tip

#### **`df.loc[rowA1, col1] = <nuevo_valor>`**
```


```{admonition} Actualización con máscara
:class: tip

#### **`df.loc[<expression>, <col1>] = <nuevo_valor>`**
```

## **3. Selección por Posición**

### **3.1. Selección de filas**

```{admonition} Seleccionar una única fila
:class: tip

#### **`df.iloc[i]`**
```

```{admonition} Seleccionar múltiples filas
:class: tip

#### **`df.iloc[[i, j, k]]`**
```

```{admonition} Seleccionar con slicing
:class: tip

#### **`df.iloc[i:k]`**
```

### **3.2. Selección de columnas**

```{admonition} Seleccionar una única columna
:class: tip

#### **`df.iloc[:, m]`**
```

```{admonition} Seleccionar múltiples columnas
:class: tip

#### **`df.iloc[:, [m, n, p]]`**
```

```{admonition} Seleccionar con slicing
:class: tip

#### **`df.iloc[:, m:p]`**
```

### **3.3. Selección de filas y columnas**

```{admonition} Seleccionar una fila y una columna
:class: tip

#### **`df.iloc[i, m]`**
```

```{admonition} Seleccionar múltiples filas y columnas
:class: tip

#### **`df.iloc[[i, j, k], [m, n, p]]`**
```

```{admonition} Seleccionar con Slicing
:class: tip

#### **`df.iloc[i:k, m:p]`**
```

### **3.4. Selección por máscara**

```{admonition} Seleccionar por máscara
:class: tip

#### **`df.iloc[<expression>]`**

Donde <expression> es una condición booleana aplicada sobre el DataFrame, evaluando cada fila y devolviendo True para las filas que se seleccionarán.
```

### **3.5. Actualización de valores**

```{admonition} Actualización de un valor
:class: tip

#### **`df.iloc[i, m] = <nuevo_valor>`**
```


```{admonition} Actualización con máscara
:class: tip

#### **`df.iloc[<expression>, <m>] = <nuevo_valor>`**
```

## **4. Selección por cuadro delimitador**

```{admonition} Actualización de un valor
:class: tip

#### **gdf.cx[xmin:xmax, ymin:ymax]**
```